In [1]:
from option import args
import torch
import utility
import data
import utils
import model
import loss
from trainer import Trainer
import os
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

np.random.seed(args.seed)
ids = utils.get_ids()
# ids.sort()
total_vols = args.no_vols+args.test_vols
ids.sort()

ids = ids[:total_vols]
ids = np.random.choice(ids,total_vols,replace = False)
print(ids)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


number of common Subjects  171
['104416' '128935' '109123' '115017' '102816' '125525' '105923' '130114'
 '111312' '102311' '118225' '100610' '131722' '130518' '114823' '126426'
 '111514' '116726' '115825' '108323']


In [2]:
args.run_name = "jupyter"
if(args.model_type == '2d'):
    args.block_size = (32,32,1)

if (args.run_name == '..'):
    args.run_name = f"{args.model},attn_{args.attention},{args.model_type},{args.encoder},{args.no_vols}_train,{args.test_vols}_test,growth{args.growth},loss_{args.loss},start_var_{args.start_var},batch_size{args.batch_size}"
else:
    args.run_name = f"{args.model},attn_{args.attention},{args.model_type},{args.encoder},{args.no_vols}_train,{args.test_vols}_test,growth{args.growth},loss_{args.loss},start_var_{args.start_var},batch_size{args.batch_size},{args.run_name}"
print(args.run_name)


dmri_rdn,attn_False,3d,rdb,10_train,10_test,growth32,loss_1*MSE,start_var_False,batch_size16,jupyter


In [3]:
logger = SummaryWriter('runs/'+ args.run_name)

checkpoint = utility.checkpoint(args)       ## setting the log and the train information
if checkpoint.ok:
    loader = data.Data(args,ids= ids)
    logger = SummaryWriter('runs/'+ args.run_name)

Loading Done


In [4]:
t = next(iter(loader.training_data))

In [5]:
rel_coord = t[3]

In [6]:
rel_coord.shape

torch.Size([1, 16, 3, 32, 4, 32])

In [11]:
rel_coord[0][:,0,0,0]

tensor([32, 50, 42])

In [11]:
rel_coord.shape

torch.Size([8, 3, 32, 4, 32])

In [ ]:
t[]

In [39]:
blk_hr = [ [i.item() for i in list(t[2][0][j]) ] for j in range(8)]
blk_lr = [ [i.item() for i in list(t[1][0][j]) ] for j in range(8)]

In [152]:
len(t[2][0])

8

In [139]:

blk_hr = [ [i.item() for i in list(t[2][0][j]) ] for j in range(8)]
blk_lr = [ [i.item() for i in list(t[1][0][j]) ] for j in range(8)]
res = []
for n in range(len(blk_hr)):
    blk_x1,blk_x2,blk_y1,blk_y2,blk_z1,blk_z2 = blk_hr[n]
    t = []
    for i in range(blk_x1,blk_x2+1):
        l = []
        for j in range(blk_y1,blk_y2+1):
            q = []
            for k in range(blk_z1,blk_z2+1):
                q.append((i,j,k))
            l.append(q)
            q = []
        t.append(l)
        l = []
    res.append(t)
            

In [153]:
res = np.asarray(res)

In [154]:
res.shape


(8, 32, 32, 4, 3)

In [155]:
b,h,w,d,c = res.shape
res = res.reshape(b,c,h,w,d)

In [161]:
res.shape

(8, 3, 32, 32, 4)

In [172]:
chk = res[0,:,0,0,3]

In [173]:
chk.shape,chk

((3,), array([64, 53, 95]))

In [52]:
np.arange(blk_hr[0][0],blk_hr[0][1]+1)x

array([64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
       81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95])

In [57]:
np.arange(blk_hr[0][2],blk_hr[0][3]+1)

array([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63])

In [58]:
np.arange(blk_hr[0][4],blk_hr[0][5]+1)

array([92, 93, 94, 95])

In [110]:
import numpy as np
x = torch.arange(64, 96, 1)
y = torch.arange(32, 64, 1)
z = torch.arange(92, 96, 1)
X= torch.meshgrid(x,y,z)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3516.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [116]:
X[0].shape

torch.Size([32, 32, 4])

In [117]:
X[1].shape

torch.Size([32, 32, 4])

In [123]:
X[1][0,:,:]

tensor([[32, 32, 32, 32],
        [33, 33, 33, 33],
        [34, 34, 34, 34],
        [35, 35, 35, 35],
        [36, 36, 36, 36],
        [37, 37, 37, 37],
        [38, 38, 38, 38],
        [39, 39, 39, 39],
        [40, 40, 40, 40],
        [41, 41, 41, 41],
        [42, 42, 42, 42],
        [43, 43, 43, 43],
        [44, 44, 44, 44],
        [45, 45, 45, 45],
        [46, 46, 46, 46],
        [47, 47, 47, 47],
        [48, 48, 48, 48],
        [49, 49, 49, 49],
        [50, 50, 50, 50],
        [51, 51, 51, 51],
        [52, 52, 52, 52],
        [53, 53, 53, 53],
        [54, 54, 54, 54],
        [55, 55, 55, 55],
        [56, 56, 56, 56],
        [57, 57, 57, 57],
        [58, 58, 58, 58],
        [59, 59, 59, 59],
        [60, 60, 60, 60],
        [61, 61, 61, 61],
        [62, 62, 62, 62],
        [63, 63, 63, 63]])

In [70]:
XY=np.array([X.flatten(),Y.flatten()]).T

In [71]:
XY.shape

(3969, 2)

In [41]:
blk_hr[0][1]

[64, 95, 32, 63, 92, 95]

In [44]:
len(np.arange(blk_hr[0][0],blk_hr[0][1]+1))

32

In [4]:
model = model.Model(args)
loss = loss.Loss(args, checkpoint)
t = Trainer(args, loader, model, loss, checkpoint,logger)

Making model... here
Preparing loss function:
1.000 * MSE


In [5]:
while not t.terminate():
    t.train()
    t.test()

Preparing loss function:
1.000 * MSE
[Epoch 0]	Learning rate: 5.00e-3


  0%|                                                                                           | 0/214 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3516.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 30%|██▋      | 33/110 [00:11<00:23,  3.34it/s, scale=[1. 1. 1.], blk_size=[16, 32, 4, 32, 7], hfen=1.13e+12, psnr=-226]

KeyboardInterrupt: 

In [ ]:
t.train()


[Epoch 1]	Learning rate: 5.00e-3



[MSE: 9139539781591087710208.0000]:  21%|▏| 45/214 [00:08<00:29,  5.67it/s, scale=[1. 1. 1.], blk_size=[16, 4, 32, 32, 7


[MSE: 33291828955897599350773121024.0000]:  54%|▌| 115/214 [00:17<00:04, 23.08it/s, scale=[1. 1. 1.], blk_size=[16, 4, 3


In [ ]:
t.test()